# Load images stored

In [1]:
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
import keras
from keras import models, Sequential
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, Dropout, Multiply
from keras.layers import AveragePooling2D, MaxPooling2D
from keras.models import Model, Sequential
import matplotlib.pyplot as plt
import time
import numpy as np
from keras.applications.resnet import ResNet50
import tensorflow as tf
from keras import losses
import keras.backend as K
import tensorflow as tf

np.random.seed(0)
tf.random.set_seed(0)

Using TensorFlow backend.


In [2]:
def load_images(train, directory):
    
    # This function loads the images, resizes them and puts them into an array
    
    img_size = 224
    train_image = []
    for name in train['image_id']:
        path = directory + 'images/' + name + '.jpg'
        img = cv2.imread(path)
        img = cv2.resize(img, (img_size, img_size))
        train_image.append(img)
    train_image_array = np.array(train_image)
    
    return train_image_array

In [3]:
def load_images_augmented(directory):
    
    # This function loads the augmented images and the augmented csv file
    df_train = pd.read_csv(directory + 'augmented.csv')
    
    train_image = []
    for name in df_train['image_id']:
        path = directory + 'images_resized_augmented/' + name + '.jpg'
        img = cv2.imread(path)
        train_image.append(img)
    train_image_array = np.array(train_image)
    
    return  train_image_array, df_train

In [4]:
directory = 'C:/Users/julen/OneDrive/Escritorio/IA/CS577-Deep-Learning/Project/'
x_train, df_train = load_images_augmented(directory)
df_test = pd.read_csv(directory + 'test.csv')
x_test = load_images(df_test, directory)

In [5]:
# # Normalize -> We don't because of memory issues. A float32 occupies 4x more than the original uint8
# x_train = x_train / 255.0 
# x_test =  x_test / 255.0

# Split training set into train and validation

In [6]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

y_train = df_train[['healthy', 'multiple_diseases', 'rust', 'scab']].to_numpy()

x_train_original, y_train_original = shuffle(x_train, y_train)

x_train, x_val, y_train, y_val = train_test_split(x_train_original, y_train_original, test_size = 0.2, random_state = 2020)

print('Size of x_train: ', x_train.shape)
print('Size of x_val: ', x_val.shape)

Size of x_train:  (7284, 224, 224, 3)
Size of x_val:  (1821, 224, 224, 3)


In [7]:
def predict_and_save(model, x_test, name):
    x_pred = model.predict(x_test)
    df_test['healthy'] = x_pred[:,0]
    df_test['multiple_diseases'] = x_pred[:,1]
    df_test['rust'] = x_pred[:,2]
    df_test['scab'] = x_pred[:,3]
    df_test.to_csv(name, index = None)

In [8]:
#See distribution of samples
y_train.sum(axis = 0)

array([2070,  346, 2495, 2373], dtype=int64)

# First approach: Batch Confusion Norm

In [9]:
def model_plot(history):

    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'validation'], loc='upper left')
    plt.show()

In [10]:
def print_score(model, x_test, y_test):
    score = model.evaluate(x_test, y_test, verbose = 0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [11]:
input_shape = (224, 224, 3)
model_resnet = ResNet50(include_top = False, weights = 'imagenet', input_shape = input_shape)

In [12]:
def ASSP(output, act_name): # TODO: Check if it is well built. Not like in paper. Maybe different image size?
    ASSP = Conv2D(filters = 2048, kernel_size = (1,1))(output)
    ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 2)(ASSP)
#     ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 4)(ASSP)
    # ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 6)(ASSP)
    # ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 7)(ASSP)

    ASSP = Activation(act_name)(ASSP)
    
    return ASSP

In [13]:


def compute_bcn_loss(y_pred):

    
    matrix = K.dot(K.transpose(y_pred), y_pred)
    eigenvalues, eigenvectors = tf.linalg.eigh(matrix)
    bcn_loss = K.sum(eigenvalues) 
    
    print(eigenvalues)
    print(type(eigenvalues))
    print(type(y_pred))
    print('y_pred shape: ', y_pred.shape)
    print('yPred.transpose shape: ', K.transpose(y_pred).shape)
    print('matrix shape: ', matrix.shape)
    print('eigenvalues shape: ', eigenvalues.shape)
    
    print()
    
    
    
    return bcn_loss



def custom_loss(y_true, y_pred):
    
    categorical_crossentropy = losses.categorical_crossentropy(y_true, y_pred)
    
    BCN_loss = compute_bcn_loss(y_pred)
    
    loss_total = categorical_crossentropy + BCN_loss
    
    return loss_total
    
    
    
    
    

In [14]:
inputs = Input(shape = (224, 224, 3))
out = model_resnet(inputs)
features = ASSP(out, 'relu')
attention = ASSP(out, 'sigmoid')
tensor_total = Multiply()([features, attention])
tensor_total_flat = Flatten()(tensor_total)
tensor_total_flat = Dense(256, activation = 'relu')(tensor_total_flat) # TODO: Check if it should be relu or another
tensor_total_flat = Dense(4, activation = 'softmax')(tensor_total_flat)

model = Model(inputs=inputs, outputs= tensor_total_flat)

model.compile(optimizer = 'adam', loss = custom_loss, metrics = ['categorical_accuracy'])

print(model.summary())

Tensor("loss/dense_2_loss/custom_loss/SelfAdjointEigV2:0", shape=(4,), dtype=float32)
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
y_pred shape:  (None, 4)
yPred.transpose shape:  (4, None)
matrix shape:  (4, 4)
eigenvalues shape:  (4,)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
resnet50 (Model)                (None, 7, 7, 2048)   23587712    input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 7, 7, 2048)   4196352     resnet50[1][0]                   

In [15]:
model.fit(x_train, y_train, batch_size = 2, epochs = 3, validation_data = (x_val, y_val))

Train on 7284 samples, validate on 1821 samples
Epoch 1/3


ResourceExhaustedError:  OOM when allocating tensor with shape[3,3,2048,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node mul_1092 (defined at C:\Programs_julen\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_50206]

Function call stack:
keras_scratch_graph


In [ ]:
# TODO: In order to set the batch size and each batch with different labels, we should use fit_generator or train_on_batch

# Simple model

In [10]:
def build_model(x_train, y_train, batch_size, epochs, verbose = True):
    
    
    ##model building
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu', input_shape = (224, 224, 3)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(1024, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dense(128, activation='relu'))

    model.add(Dense(4, activation='softmax'))
    

    print(model.summary())
    start_time = time.time()
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam' , metrics = ['categorical_accuracy'])
    history = model.fit(x = x_train, y = y_train, validation_data = (x_val, y_val), batch_size = batch_size, epochs = epochs, verbose = verbose)
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return (model, history)

In [14]:
model_old, history = build_model(x_train, y_train, batch_size = 16, epochs = 1)
model_plot(history)
print_score(model_old, x_test, y_test)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 54, 54, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 52, 52, 512)      

KeyError: 'acc'

In [18]:
df_test['healthy'] = x_pred[:,0]
df_test['multiple_diseases'] = x_pred[:,1]
df_test['rust'] = x_pred[:,2]
df_test['scab'] = x_pred[:,3]
df_test.to_csv('FGVC_submission_.csv', index = None)

In [17]:
df_test

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.205251,0.045785,0.368202,0.380761
1,Test_1,0.231453,0.047746,0.345878,0.374923
2,Test_2,0.223623,0.071676,0.342834,0.361867
3,Test_3,0.308657,0.102331,0.295816,0.293197
4,Test_4,0.230924,0.044711,0.364794,0.359571
...,...,...,...,...,...
1816,Test_1816,0.220903,0.053204,0.343984,0.381910
1817,Test_1817,0.273419,0.108838,0.302602,0.315141
1818,Test_1818,0.324635,0.099112,0.290823,0.285430
1819,Test_1819,0.337709,0.099663,0.284925,0.277703


# Second approach: Localization, Pooling and Embedding

In [2]:
from keras import backend as K
from keras.layers import Layer, InputSpec
from keras.legacy import interfaces




class GlobalKMaxPooling2D(Layer): #Inherits the properties of Layer class
    """K Max Pooling operation for spatial data.
    
    # Arguments
        
        data_format: A string,
            one of `"channels_last"` (default) or `"channels_first"`.
            The ordering of the dimensions in the inputs.
            `"channels_last"` corresponds to inputs with shape
            `(batch, height, width, channels)` while `"channels_first"`
            corresponds to inputs with shape
            `(batch, channels, height, width)`.
            It defaults to the `image_data_format` value found in your
            Keras config file at `~/.keras/keras.json`.
            If you never set it, then it will be `"channels_last"`.
            
        K: An Integer,
            states the number of selected maximal values over which the
            average is going to be computed.
            
            
    # Input shape
    
        - If `data_format='channels_last'`:
            4D tensor with shape:
            `(batch_size, rows, cols, channels)`
            
        - If `data_format='channels_first'`:
            4D tensor with shape:
            `(batch_size, channels, rows, cols)`
            
    # Output shape
    
        - If `data_format='channels_last'`:
            4D tensor with shape:
            `(batch_size, pooled_rows, pooled_cols, channels)`
            
        - If `data_format='channels_first'`:
            4D tensor with shape:
            `(batch_size, channels, pooled_rows, pooled_cols)`    
    
    
    """
    

    def __init__(self, data_format=None, k = 10, **kwargs):
        super(GlobalKMaxPooling2D, self).__init__(**kwargs)
        self.data_format = K.normalize_data_format(data_format)
        self.input_spec = InputSpec(ndim=4)
        self.k = k

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_last':
            return (input_shape[0], input_shape[3])
        else:
            return (input_shape[0], input_shape[1])

    def get_config(self):
        config = {'data_format': self.data_format, 'k' : self.k}
        base_config = super(GlobalKMaxPooling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
        
    def call(self, inputs):
        if self.data_format == 'channels_last':
            # Here first sort
            # Then take K maximum values
            # Then average them
            k = self.k

            input_reshaped = tf.reshape(inputs, [tf.shape(inputs)[0], -1, tf.shape(inputs)[3]])
            input_reshaped = tf.reshape(input_reshaped, [tf.shape(input_reshaped)[0], tf.shape(input_reshaped)[2], tf.shape(input_reshaped)[1]])
            top_k = tf.math.top_k(input_reshaped, k=k, sorted = True, name = None)[0]
            mean = tf.keras.backend.mean(top_k, axis = 2)
            #assert ((input_reshaped.get_shape()[0], input_reshaped.get_shape()[-1]) == mean.get_shape())
        
        return mean

    
    
    

In [3]:
def build_lrfn(lr_start=0.00001, lr_max=0.00005, 
               lr_min=0.00001, lr_rampup_epochs=5, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max #* strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) *\
                 lr_exp_decay**(epoch - lr_rampup_epochs\
                                - lr_sustain_epochs) + lr_min
        return lr
    return lrfn

In [4]:
lrfn = build_lrfn()
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)

In [5]:
import keras
from keras import models, Sequential
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, Dropout, Multiply
from keras.layers import AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Reshape
from keras.models import Model, Sequential
import matplotlib.pyplot as plt
import time
import numpy as np
from keras.applications.resnet import ResNet50

In [6]:
input_shape = (224, 224, 3)
model_resnet = ResNet50(include_top = False, weights = 'imagenet', input_shape = input_shape)

In [53]:

model = Sequential()
model.add(model_resnet)
model.add(GlobalKMaxPooling2D(data_format = 'channels_last' , k = 4))
# model.add(Dense(128, activation = 'relu'))
model.add(Dense(4, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = EmbeddingLayerLoss(model.layers[2]), metrics = ['categorical_accuracy'])
print(model.summary())


NameError: name 'EmbeddingLayerLoss' is not defined

In [ ]:
model.layers[-2]


In [ ]:
y_train.shape

In [431]:
from keras.applications.resnet import ResNet50
from keras.losses import categorical_crossentropy
import keras
from keras import models, Sequential
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, Dropout, Multiply
from keras.layers import AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Reshape
from keras.models import Model, Sequential
import matplotlib.pyplot as plt
import time

tf.compat.v1.enable_eager_execution()
print(tf.executing_eagerly())
    
class ElopeModel(object):
    
    def __init__(self):
        
        # Model
        self.model = self.build_model()
        # Print a model summary
        self.model.summary()
        
        # Optimizer
        self.optimizer = 'Adam'
        
        # Have to convert to tensor??
        self.loss_parameters = {'means' : 0, 'lr' : 0.5, 'landa' : 2.0,
                               'gamma' : 16.0, 'm' : 0.75}

        
        #Loss Function
        self.loss_func = self.model_loss()
        
        self.compile()
        
        
    def build_model(self):
        tf.compat.v1.enable_eager_execution()
        print('Eager execution:', tf.executing_eagerly())
        input_shape = (224, 224, 3)
        model_resnet = ResNet50(include_top = False, weights = 'imagenet', input_shape = input_shape)
        model = Sequential()
        
        model.add(model_resnet)
        model.add(GlobalKMaxPooling2D(data_format = 'channels_last' , k = 4))
        model.add(Dense(4, activation = 'softmax'))
        tf.compat.v1.enable_eager_execution()
        print('Eager execution:', tf.executing_eagerly())
        
        return model
    
    
    def calculate_within_class_loss(self, y_true, y_pred, fx_tensor): #fx_tensor = Output of layer
       
        mean_tensor = self.loss_parameters['means']
        alpha = self.loss_parameters['lr']

        dimension_tensor = K.shape(fx_tensor)[1]
        num_classes_tensor = K.shape(y_true)[1]
        if isinstance(mean_tensor, int): # Initialize them if they are not yet initialized
            mean_tensor = tf.zeros([num_classes, dimension], dtype = tf.dtypes.float64)
        num_samples_tensor = K.shape(fx_tensor)[0]
        
        # Ensure they are float 64
        fx_tensor = tf.cast(fx_tensor, dtype = tf.dtypes.float64)
        

        fx_expanded = tf.broadcast_to(tf.expand_dims(fx_tensor, axis = -1), [num_samples_tensor, dimension_tensor, num_classes_tensor])
        y_true_expanded = tf.broadcast_to(tf.expand_dims(y_true_tensor, axis = 1), [num_samples_tensor, dimension_tensor, num_classes_tensor])
        mean_expanded = tf.broadcast_to(tf.expand_dims(mean_tensor, axis = 0), [num_samples_tensor, num_classes_tensor, dimension_tensor])

        mean_expanded = tf.transpose(mean_expanded, perm = [0,2,1])

        up = tf.reduce_sum(tf.multiply(tf.subtract(mean_expanded, fx_expanded), tf.cast(y_true_expanded, dtype = tf.dtypes.float64)), axis = 0)
        y_true_cut = tf.reduce_sum(y_true_tensor, axis = 0)
        down = tf.add(y_true_cut, tf.constant(1))
        down = tf.cast(down, dtype = tf.dtypes.float64)
        delta = tf.divide(up, down)
        delta = tf.transpose(delta)

        mean_new = tf.subtract(mean_tensor, tf.scalar_mul(tf.constant(alpha, dtype = tf.dtypes.float64), delta))

        # Now calculate the loss
        mean_new_expanded = tf.broadcast_to(tf.expand_dims(mean_new, axis = 0), [num_samples_tensor, num_classes_tensor, dimension_tensor])
        mean_new_expanded = tf.transpose(mean_new_expanded,  perm = [0,2,1]) 
        inside = tf.reduce_sum(tf.multiply(tf.subtract(fx_expanded, mean_new_expanded), tf.cast(y_true_expanded, dtype = tf.dtypes.float64)), axis = 2)


        tot = tf.reduce_sum(tf.multiply(inside, inside)) # Apply the norm
        down = tf.multiply(tf.constant(2), num_samples_tensor)
        tot = tf.divide(tot, tf.cast(down, dtype = tf.dtypes.float64))

        mean_tensor = mean_new
        
        # N = number of samples in the batch
        # f(xn) -> Dimension = 2048 (not the batch because it it xn, only 1 sample)
        # class means -> should be the same as f(xn) x number of classes
        # Therefore ||f(xn) - U(cn)||^2 will be a number, because f(xn) is a vector, not a matrix

        
        return loss
    
    
    
    def calculate_between_class_loss(self, y_true, y_pred, layer_output):
        # Layer there or in self??
        class_means = self.loss_parameters['means']
        gamma = self.loss_parameters['gamma']
        m = self.loss_parameters['m']
        # m -> Margin
        # P -> class-pairs in the current batch
        # |P| -> Cardinality of P, number of elements in the set
        
#         bc_loss = 
        
        return None
    
    
    
    
    def model_loss(self):
        """" Wrapper function which calculates auxiliary values for the complete loss function.
         Returns a *function* which calculates the complete loss given only the input and target output """
        # This part has to be developed
        
        #Within class loss
        within_class_loss_func = self.calculate_within_class_loss
        
        # Between class loss
        between_class_loss_func = self.calculate_between_class_loss
        
        lay_out = self.model.layers[-2].output
        
        
        def ElopeLoss(y_true, y_pred):
            
            # Within Class loss has to be computed first, in order to get the new class means updated
            
            within_class_loss = within_class_loss_func(y_true, y_pred, lay_out)
            
            between_class_loss = between_class_loss_func(y_true, y_pred, lay_out)
            
            cat_cross_loss = categorical_crossentropy(y_true, y_pred)
            
            model_loss = cat_cross_loss + within_class_loss #+ between_class_loss
            
            return model_loss
        
        
        return ElopeLoss
    
    
    def compile(self):
        """ Compiles the Keras model. Includes metrics to differentiate between the two main loss terms """
        tf.compat.v1.enable_eager_execution()
        print('Eager execution:', tf.executing_eagerly())
        self.model.compile(optimizer = self.optimizer, loss = self.loss_func,
                          metrics = [categorical_crossentropy]) # Here put the two losses better
        print('Model Compiled!')
        
    
    def load_trained_weights(self, weights):
        """ Loads weights of a pre-trained model. 'weights' is path to h5 model\weights file"""
        self.model.load_weights(weights)
        print('Weights from {} loaded successfully'.format(weights))
        

True


In [432]:
Elope = ElopeModel()

Eager execution: True
Eager execution: True
Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_k_max_pooling2d_31 (G (None, 2048)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 4)                 8196      
Total params: 23,595,908
Trainable params: 23,542,788
Non-trainable params: 53,120
_________________________________________________________________
Eager execution: True
Model Compiled!


In [33]:
print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

AttributeError: module 'tensorflow' has no attribute 'VERSION'

In [9]:
tf.executing_eagerly()

True

In [32]:
history = Elope.model.fit(x_train, y_train, batch_size = 12, epochs = 1, 
                    validation_data = (x_val, y_val), callbacks = [lr_schedule])

Train on 7284 samples, validate on 1821 samples
Epoch 1/1

Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
7284/7284 [==============================] - 471s 65ms/step - loss: 1.6179 - categorical_crossentropy: 1.6179 - val_loss: 0.9083 - val_categorical_crossentropy: 0.9083


# Steps needed for calculating the Loss

In [168]:
# First we are going to specify some random values and convert them to tensors

f_x = np.random.rand(3,2048)
mean = np.random.rand(4,2048)
y_true = np.array([[0,0,1,0], [1,0,0,0], [0,1,0,0]])

fx_tensor = tf.convert_to_tensor(f_x)
mean_tensor = tf.convert_to_tensor(mean)
y_true_tensor = tf.convert_to_tensor(y_true)

In [169]:
fx_tensor.shape[1]

2048

In [170]:
# Now we are going to get the necessary slices from the tensors

fx_tensor_slice = tf.slice(f_x, [1,0], [1, tf.shape(fx_tensor)[1].numpy()]) #Begin, size
y_true_slice = tf.slice(y_true, [1,0], [1,4])
class_number = np.argmax(y_true_slice.numpy()) # Class goes from 0 to 3, in order to fit better into slice method
mean_slice = tf.slice(mean_tensor, [class_number, 0], [1,tf.shape(mean_tensor)[1].numpy()])

In [171]:
print('f_x shape: ', tf.shape(fx_tensor_slice))
print('mean shape: ', tf.shape(mean_slice) )
print('y_true shape: ', tf.shape(y_true_slice))

f_x shape:  tf.Tensor([   1 2048], shape=(2,), dtype=int32)
mean shape:  tf.Tensor([   1 2048], shape=(2,), dtype=int32)
y_true shape:  tf.Tensor([1 4], shape=(2,), dtype=int32)


In [172]:
mean_slice - fx_tensor_slice

<tf.Tensor: shape=(1, 2048), dtype=float64, numpy=
array([[ 0.19938234,  0.00816488, -0.6013304 , ...,  0.21713867,
        -0.3828902 , -0.39345389]])>

In [173]:
K.eval(fx_tensor)

array([[0.81525007, 0.79134176, 0.44398508, ..., 0.6280034 , 0.2294754 ,
        0.76706728],
       [0.43227968, 0.8426107 , 0.91554432, ..., 0.17903316, 0.97670381,
        0.77232114],
       [0.14680833, 0.40305313, 0.5662111 , ..., 0.21604621, 0.48619845,
        0.07001679]])

# Calculate loss with graph mode

In [426]:
alpha = 0.5
num_classes = mean_tensor.shape[0]
num_samples_in_batch = fx_tensor.shape[0]
dimensions = mean_tensor.shape[1]

# print('mean_tensor shape: ', mean_tensor.shape)
# print('fx_tensor shape: ', fx_tensor.shape)
# print('y_true shape: ', y_true.shape)

dimension_tensor = K.shape(fx_tensor)[1]
num_classes_tensor = K.shape(y_true)[1]
num_samples_tensor = K.shape(y_true)[0]

fx_expanded = tf.broadcast_to(tf.expand_dims(fx_tensor, axis = -1), [num_samples_tensor, dimension_tensor, num_classes_tensor])
y_true_expanded = tf.broadcast_to(tf.expand_dims(y_true_tensor, axis = 1), [num_samples_tensor, dimension_tensor, num_classes_tensor])
mean_expanded = tf.broadcast_to(tf.expand_dims(mean_tensor, axis = 0), [num_samples_tensor, num_classes_tensor, dimension_tensor])
# print(mean_expanded.numpy())
mean_expanded = tf.transpose(mean_expanded, perm = [0,2,1])
# mean_expanded = tf.reshape(mean_expanded, [num_samples_tensor, dimension_tensor, num_classes_tensor])

# print('fx_expanded shape: ', fx_expanded.shape)
# print('y_true_expanded shape: ', y_true_expanded.shape)
# print('mean_expanded shape: ', mean_expanded.shape)
# print()
# print('fx_expanded: ', fx_expanded.numpy())
# print()
# print('y_true_expanded: ', y_true_expanded.numpy())
# print()
# print('mean_expanded: ', mean_expanded.numpy())
# print()
# print('result of subtraction: ', tf.multiply(tf.subtract(mean_expanded, fx_expanded), tf.cast(y_true_expanded, dtype = tf.dtypes.float64)).numpy())

# up = (mean_expanded - fx_expanded) * y_true_expanded (acting as mask)
up = tf.reduce_sum(tf.multiply(tf.subtract(mean_expanded, fx_expanded), tf.cast(y_true_expanded, dtype = tf.dtypes.float64)), axis = 0)
# print('upper part: ', up.numpy())
# for the part below:
y_true_cut = tf.reduce_sum(y_true_tensor, axis = 0)
down = tf.add(y_true_cut, tf.constant(1))
down = tf.cast(down, dtype = tf.dtypes.float64)
delta = tf.divide(up, down)

# print(mean_tensor.numpy() - 0.5 * delta.numpy())
delta = tf.transpose(delta)
# print(delta.numpy())
# print(mean_tensor.numpy())
# print('delta shape: ', delta.shape)
# print('mean shape:', mean_tensor.shape)
mean_new = tf.subtract(mean_tensor, tf.scalar_mul(tf.constant(alpha, dtype = tf.dtypes.float64), delta))

mean_new_sum = np.sum(mean_new.numpy())
print()
print(mean_new_sum)
print(mean_new.numpy())
print()
print('---------------- NOW THE LOSS ---------------')
print()

# Now calculate the loss
mean_new_expanded = tf.broadcast_to(tf.expand_dims(mean_new, axis = 0), [num_samples_tensor, num_classes_tensor, dimension_tensor])
mean_new_expanded = tf.transpose(mean_new_expanded,  perm = [0,2,1]) 
# print(mean_new_expanded.numpy())
print()
# mean_new_expanded = tf.reshape(mean_new_expanded, [num_samples_tensor, dimension_tensor, num_classes_tensor])
inside = tf.reduce_sum(tf.multiply(tf.subtract(fx_expanded, mean_new_expanded), tf.cast(y_true_expanded, dtype = tf.dtypes.float64)), axis = 2)

# print(inside.shape)
# print()
# print(inside.numpy())
tot = tf.reduce_sum(tf.multiply(inside, inside)) # Apply the norm
down = tf.multiply(tf.constant(2), num_samples_tensor)
tot = tf.divide(tot, tf.cast(down, dtype = tf.dtypes.float64))
# print(tot.shape)
print(tot.numpy())

# mean_tensor = mean_new


13.666666666666664
[[-1.75       -0.75       -2.75       -2.25      ]
 [ 3.83333333  4.83333333  6.66666667  5.83333333]]

---------------- NOW THE LOSS ---------------


3.550925925925926


## Implement it in Numpy to see it is correct

In [419]:
# First we are going to specify some random values and convert them to tensors

# f_x = np.random.rand(6,2048) 
# mean = np.random.rand(4,2048) 
# y_true = np.array([[0,0,1,0], [1,0,0,0], [0,1,0,0], [0,0,0,1], [1,0,0,0], [0,1,0,0]])

f_x = np.array([[2,3,4,5], [-1, 0, -2, -3], [5,6, 8, 6]], dtype = 'float64')
mean = np.array([[-2, -1, -3, -2], [4, 5, 7, 6]], dtype = 'float64')
y_true = np.array([[0,1], [1,0], [0,1]])

fx_tensor = tf.convert_to_tensor(f_x, dtype = tf.dtypes.float64)
mean_tensor = tf.convert_to_tensor(mean, dtype = tf.dtypes.float64)
y_true_tensor = tf.convert_to_tensor(y_true)


In [423]:
# This is well done!
alpha = 0.5
num_classes = 2
num_samples_in_batch = 3
dimension = 4

# LOOP THROUGH CLASSES TO CALCULATE NEW MEAN
for i in range(0, num_classes): # Loops throgh the classes

    counter = 1 # to divide it
    summ = np.zeros(dimension)  # Initialize class means 
    mean_i = mean[i]
    for j in range(0, num_samples_in_batch): # Loops through the images in the batch  
        y_true_sample = y_true[j] # Class of the i image
        class_number = np.argmax(y_true_sample) 
        
        if class_number == i: # if not, do nothing
            counter += 1
            summ = summ + (mean_i - f_x[j])
        
    total = summ / counter
    mean[i] = mean_i - alpha*total 
    
print('The new mean: ', mean)
print()

# Calculate the loss
suma = 0
for i in range(0, num_samples_in_batch):
    y_true_sample = y_true[i] # Class of the i image
    class_number = np.argmax(y_true_sample)
    inside = f_x[i] - mean[class_number]
    norm = np.linalg.norm(inside)
    squared = np.square(norm)
    suma = suma + np.square(np.linalg.norm(f_x[i] - mean[class_number]))
loss = suma / (2*num_samples_in_batch)
print('The final loss:', loss)


The new mean:  [[-1.31640625 -0.31640625 -2.31640625 -2.68359375]
 [ 3.59876543  4.59876543  6.19753086  5.59876543]]

The final loss: 3.006169368040572


In [348]:
mean[0] = np.array([5,3,4,5])
mean[0]

array([5, 3, 4, 5])

In [308]:
23/6 + 29/6  + 40/6 + 35/6 -7/4 -3/4 -11/4 -1/4

15.666666666666664

In [309]:
-7/4 -3/4 -11/4 -1/4

-5.5

In [338]:
np.array([-2, -1, -3, -2]) - np.array([-0.25, -0.25, -0.25, 0.25])

array([-1.75, -0.75, -2.75, -2.25])

In [252]:
# Complete method

# CANNOT USE NUMPY OR ARRAYS! Because it would make problems for backpropagation, as they are not in the graph.
# Other way is to compute the backpropagation by ourselves.

alpha = 0.5
mean_new = [[], [], [], []] # 4 classes
num_classes = tf.shape(mean_tensor)[0].numpy()
num_samples_in_batch = tf.shape(fx_tensor)[0].numpy()
dimension = tf.shape(mean_tensor)[1].numpy()

# LOOP THROUGH CLASSES TO CALCULATE NEW MEAN
for i in range(0, num_classes): # Loops throgh the classes

    counter = 1 # to divide it
    summ = tf.zeros(dimension, dtype = tf.dtypes.float64) # Initialize class means 
    # TODO: Initialize class means outside this function, as a parameter of self
    mean_slice = tf.slice(mean_tensor, [i, 0], [1,dimension])

    for j in range(0, num_samples_in_batch): # Loops through the images in the batch  
        y_true_slice = tf.slice(y_true, [j,0], [1,4]) # Class of the i image
        class_number = np.argmax(y_true_slice.numpy()) # Class goes from 0 to 3, in order to fit better into slice method.
                                                        # Check if this is possible in the GPU

        if class_number == i: # if not, do nothing
            counter += 1
            fx_tensor_slice = tf.slice(fx_tensor, [j,0], [1, dimension]) #Begin, size
            summ = tf.add(summ, tf.subtract(mean_slice, fx_tensor_slice))
        
    total = tf.divide(summ, tf.constant(counter, dtype = tf.dtypes.float64))
#     assert(np.sum(total.numpy()) == np.sum(summ.numpy() / counter))
    
    mean_new[i] = tf.subtract(mean_slice, tf.scalar_mul(tf.constant(alpha, dtype = tf.dtypes.float64), total))
#     assert(np.sum(mean_new[i].numpy()) == np.sum(mean_slice.numpy() - alpha * total.numpy()))
    
# Assembly the new mean tensor
mean_tensor_out = tf.concat([mean_new[0], mean_new[1], mean_new[2], mean_new[3]], axis = 0)
print('mean_tensor output:', np.sum(mean_tensor_out.numpy()) )
    
    
    
    
# # CALCULATE LOSS
loss = tf.Variable(0, dtype = tf.dtypes.float64)
for i in range (0, num_samples_in_batch):
    y_true_slice = tf.slice(y_true, [i,0], [1,4]) # Take the label of the ith sample
    class_number = np.argmax(y_true_slice.numpy()) # Convert it to integer
    mean_slice = tf.slice(mean_tensor, [class_number, 0], [1, dimension])
    fx_tensor_slice = tf.slice(fx_tensor, [j,0], [1, dimension])
    # Now do the subtract, square, sum and divide
    assert(np.sum(tf.shape(mean_slice).numpy()) == np.sum(tf.shape(fx_tensor_slice).numpy()))
    rest = tf.subtract(fx_tensor_slice, mean_slice)
    norm = tf.square(tf.norm(rest))
    loss = tf.add(loss, norm)
    
loss = tf.divide(loss, tf.Variable(2*num_samples_in_batch, dtype = tf.dtypes.float64))
loss.numpy()

mean_tensor output: 4045.4047719107616


111.98640597369022

In [182]:
fx_tensor

<tf.Tensor: shape=(3, 2048), dtype=float64, numpy=
array([[0.81525007, 0.79134176, 0.44398508, ..., 0.6280034 , 0.2294754 ,
        0.76706728],
       [0.43227968, 0.8426107 , 0.91554432, ..., 0.17903316, 0.97670381,
        0.77232114],
       [0.14680833, 0.40305313, 0.5662111 , ..., 0.21604621, 0.48619845,
        0.07001679]])>

In [32]:
tf.Tensor()

TypeError: __init__() missing 3 required positional arguments: 'op', 'value_index', and 'dtype'

In [9]:
mean_tot

NameError: name 'mean_tot' is not defined

In [94]:
print('f_x shape: ', tf.shape(fx_tensor_slice))
print('mean shape: ', tf.shape(mean_slice) )
print('y_true shape: ', tf.shape(y_true_slice))

f_x shape:  tf.Tensor([   1 2048], shape=(2,), dtype=int32)
mean shape:  tf.Tensor([   1 2048], shape=(2,), dtype=int32)
y_true shape:  tf.Tensor([1 4], shape=(2,), dtype=int32)


In [37]:
mean_tensor_broadcasted = tf.broadcast_to(mean_tensor, [tf.shape(f_x_tensor)[0].numpy(), tf.shape(mean_tensor)[1].numpy()])

In [39]:
tf.math.subtract(mean_tensor_broadcasted, f_x_tensor)

<tf.Tensor: shape=(3, 2048), dtype=float64, numpy=
array([[ 0.52522555,  0.23903976,  0.00548459, ...,  0.69942552,
         0.43518934,  0.03350187],
       [ 0.18296376, -0.19688141, -0.83012728, ...,  0.15724875,
        -0.33678901, -0.52118118],
       [-0.10041236,  0.0561405 , -0.92195231, ...,  0.22434052,
         0.45476284,  0.12300551]])>

In [46]:
y_true = np.array([[0,0,1,0], [1,0,0,0], [0,1,0,0]])

In [47]:
y_true.shape

(3, 4)

In [49]:
y_true_tensor = tf.convert_to_tensor(y_true)
y_true_tensor = tf.expand_dims(y_true_tensor, -1)

In [50]:
tf.shape(y_true_tensor)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([3, 4, 1])>

In [82]:
y_true_slice.numpy()

array([[1, 0, 0, 0]])

In [83]:
np.argmax(y_true_slice.numpy())

0

In [26]:
predict_and_save(model, x_test, name = 'FGVC_submission_2_approach.csv')

In [33]:
lay_output = Elope.model.layers[-2].output

In [34]:
y_pred = tf.convert_to_tensor(Elope.model.predict(x_test))

In [41]:
y_true_tensor = tf.convert_to_tensor(y_train)

In [42]:
lay_output

<tf.Tensor 'global_k_max_pooling2d_5/Mean:0' shape=(None, None) dtype=float32>

In [43]:
y_true_tensor

<tf.Tensor: shape=(7284, 4), dtype=int64, numpy=
array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       ...,
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0]], dtype=int64)>